In [1]:
# Import the dependencies

import pandas as pd
import gmaps
import requests

# Import API key

from config import g_key

In [2]:
# Store the CSV into DataFrame

city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,2022-01-31 19:21:05,-46.1927,168.8643,59.49,91,29,1.48
1,1,Lorengau,PG,2022-01-31 19:21:06,-2.0226,147.2712,78.21,90,100,14.43
2,2,Iraray,PH,2022-01-31 19:21:06,8.9936,118.0511,77.85,87,23,7.49
3,3,Iralaya,HN,2022-01-31 19:21:07,15.0000,-83.2333,84.29,60,64,12.59
4,4,New Norfolk,AU,2022-01-31 19:21:07,-42.7826,147.0587,68.76,75,5,2.21


In [3]:
# Check data types

city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use google API key

gmaps.configure(api_key = g_key)

In [5]:
# Get the maximum temperature to fix invalid weight exception for negative max temps

max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
# Heatmap of temperature

# Get the latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature
max_temp = city_data_df["Max Temp"]

# Assign the figure variable
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights = [max(temp, 0) for temp in max_temp], 
                                 dissipating = False, max_intensity = 300, point_radius = 4)

# Assign the heatmap layer
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity

locations = city_data_df[["Lat", "Lng"]]

humidity = city_data_df["Humidity"]

fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 300, point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent cloudiness

locations = city_data_df[["Lat", "Lng"]]

clouds = city_data_df["Cloudiness"]

fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights = clouds, dissipating = False, max_intensity = 300, point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of wind speed

locations = city_data_df[["Lat", "Lng"]]

wind = city_data_df["Wind Speed"]

fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights = wind, dissipating = False, max_intensity = 300, point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Ask the customer to add a minimum and maximum temperature value

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# Filter the dataset to find the cities that fit the criteria

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()

preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Lorengau,PG,2022-01-31 19:21:06,-2.0226,147.2712,78.21,90,100,14.43
2,2,Iraray,PH,2022-01-31 19:21:06,8.9936,118.0511,77.85,87,23,7.49
3,3,Iralaya,HN,2022-01-31 19:21:07,15.0000,-83.2333,84.29,60,64,12.59
7,7,Rikitea,PF,2022-01-31 19:18:30,-23.1203,-134.9692,77.27,65,100,4.34
8,8,Kapaa,US,2022-01-31 19:21:09,22.0752,-159.3190,75.18,79,0,8.05
10,10,Hermanus,ZA,2022-01-31 19:21:10,-34.4187,19.2345,78.91,76,43,15.26
14,14,Camapua,BR,2022-01-31 19:21:12,-19.5314,-54.0439,75.81,87,100,9.78
19,19,Puerto Ayora,EC,2022-01-31 19:19:58,-0.7393,-90.3518,78.76,90,60,5.01
22,22,Vaini,TO,2022-01-31 19:21:14,-21.2000,-175.2000,79.68,75,100,14.03
24,24,Kuching,MY,2022-01-31 19:21:15,1.5500,110.3333,75.42,94,40,0.00


In [17]:
# Check for null values

preferred_cities_df.count()

City_ID       169
City          169
Country       169
Date          169
Lat           169
Lng           169
Max Temp      169
Humidity      169
Cloudiness    169
Wind Speed    169
dtype: int64

In [18]:
# Create DataFrame to store hotel names with city, country, max temp, coordinates

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Lorengau,PG,78.21,-2.0226,147.2712,
2,Iraray,PH,77.85,8.9936,118.0511,
3,Iralaya,HN,84.29,15.0000,-83.2333,
7,Rikitea,PF,77.27,-23.1203,-134.9692,
8,Kapaa,US,75.18,22.0752,-159.3190,
10,Hermanus,ZA,78.91,-34.4187,19.2345,
14,Camapua,BR,75.81,-19.5314,-54.0439,
19,Puerto Ayora,EC,78.76,-0.7393,-90.3518,
22,Vaini,TO,79.68,-21.2000,-175.2000,
24,Kuching,MY,75.42,1.5500,110.3333,


In [20]:
# Set parameters to search for a hotel

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [22]:
# Iterate through the dataframe

for index, row in hotel_df.iterrows():
    
    # retrieve lat/lng
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and lat/lng
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get JSON data from search
    
    hotels = requests.get(base_url, params = params).json()
    
    # Grab the first hotel from the results and store the name
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [23]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Lorengau,PG,78.21,-2.0226,147.2712,Lorengau Harbourside Hotel
2,Iraray,PH,77.85,8.9936,118.0511,
3,Iralaya,HN,84.29,15.0000,-83.2333,Iglesia morava renovada
7,Rikitea,PF,77.27,-23.1203,-134.9692,People ThankYou
8,Kapaa,US,75.18,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Hermanus,ZA,78.91,-34.4187,19.2345,Misty Waves Boutique Hotel
14,Camapua,BR,75.81,-19.5314,-54.0439,Comercial Estrela
19,Puerto Ayora,EC,78.76,-0.7393,-90.3518,Finch Bay Galapagos Hotel
22,Vaini,TO,79.68,-21.2000,-175.2000,Keleti Beach Resort
24,Kuching,MY,75.42,1.5500,110.3333,Hilton Kuching


In [25]:
# Add a heatmap of temperature for the vacation spots and markers for each city

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating = False, max_intensity = 300, point_radius = 4)

marker_layer = gmaps.marker_layer(locations)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the dataframe row

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))